In [1]:
import codecs
import json
import os
import random


import spacy
from spacy.tokens import DocBin, Doc
from spacy.training import Example

In [2]:
def read_menota_annotations_for_spacy():
    l = []
    with codecs.open(os.path.join("data-menota-spacy.json"), "r", encoding="utf-8") as f:
        for book in f:
            # print("book")
            for line in json.loads(book):
                # print(line)
                # print("\n")
                if all([i == "-" for i in line[0]]):
                    continue
                l.append(line)
    return l

In [3]:
data = read_menota_annotations_for_spacy()

In [4]:
data

[['í',
  'vígskǫrðum',
  'verja',
  ',',
  'þá',
  'er',
  'gott',
  'at',
  'gera',
  'hengivígskǫ',
  'af',
  'léttum',
  'viði'],
 {'pos': ['xAP',
   'xNC gN nP cD sI',
   'xVB fI tPS vA',
   '-',
   'xAV rP',
   'xVB fF tPS mIN p3 nS vA',
   'xAJ rP gN nS cN sI',
   'xIM',
   'xVB fI tPS vA',
   'xNC gN nP cA sI',
   'xAP',
   'xAJ rP gM nS cD sI',
   'xNC gM nS cD sI']}]

In [5]:
def create_vocab(training_data):
    vocabulary_set = set()
    for text, _ in training_data:
        for lemma in text:
            vocabulary_set.add(lemma)
    return vocabulary_set


def reduce_tags(pos_tag: str) -> str:
    first_part = pos_tag.strip().split(" ")[0]
    if "|" in first_part:
        return reduce_tags(first_part.split("|")[0])
    if first_part.startswith("x"):
        return first_part[1:]
    elif "00000" == first_part:
        return ""
    else:
        return pos_tag


def from_text_annotations_to_spacy_training_data(data):
    l = []
    for sentence, annotation in data:
        try:
            l.append(([token.lower() for token in sentence],
                       dict(pos=[reduce_tags(tag) for tag in annotation["pos"]])))
            # lemmas=annotation["lemmas"]
        except AttributeError:
            print(sentence)
            print(annotation)
    return l

In [6]:
pos_training_data = from_text_annotations_to_spacy_training_data(data)

In [7]:
pos_training_data[0]

(['í',
  'vígskǫrðum',
  'verja',
  ',',
  'þá',
  'er',
  'gott',
  'at',
  'gera',
  'hengivígskǫ',
  'af',
  'léttum',
  'viði'],
 {'pos': ['AP',
   'NC',
   'VB',
   '-',
   'AV',
   'VB',
   'AJ',
   'IM',
   'VB',
   'NC',
   'AP',
   'AJ',
   'NC']})

In [23]:
def create_training(training_data):
    vocab = spacy.Vocab(string=create_vocab(training_data))
    nlp = spacy.blank("is")
    db = DocBin()  # create a DocBin object
    text = []
    for item in training_data:
        words = item[0]
        pos = item[1]
#         example = Example.from_dict(Doc(vocab, words=words), pos)

        try:
            print(" ".join(words))
            print(pos)
            example = Example.from_dict(nlp.make_doc(" ".join(words)), pos)
            db.add(example.reference)
            text.append(" ".join(words))
        except ValueError:
            print("problème")
            
    return db, " ".join(text)


def save_training_data(training_data):
    random.shuffle(training_data)
    valid = training_data[:500]
    train = training_data[500:]

    train, train_corpus = create_training(train)
    train.to_disk("./data/train.spacy")
    with codecs.open("./data/train_corpus.txt", "w", encoding="utf-8") as f:
        f.write(train_corpus)

    valid, valid_corpus = create_training(valid)
    valid.to_disk("./data/valid.spacy")
    with codecs.open("./data/valid_corpus.txt", "w", encoding="utf-8") as f:
        f.write(valid_corpus)

In [24]:
save_training_data(pos_training_data)

þeir verjask vel
{'pos': ['DD', 'VB', 'AV']}
um þessi tíðendi talar meistari galterus í bók sinni
{'pos': ['AP', 'DD', 'NC', 'VB', 'NC', 'UA', 'AP', 'NC', 'DP']}
þá vil ek bjóða þér hólmgǫngu ok þat með , at vér berimsk hér á þinginu
{'pos': ['AV', 'VB', 'PE', 'VB', 'PE', 'NC', 'CC', 'DD', 'VP', '-', 'CS', 'PE', 'VB', 'AV', 'AP', 'NC']}
en þǽr eru nauðsynjar , ef skipsháski -
{'pos': ['CC', 'DD', 'VB', 'NC', '-', 'CS', 'NC', '-']}
verða þeir þá skeinisamastir
{'pos': ['VB', 'DD', 'AV', 'AJ']}
eptir um morguninn , þá er konungr klæddisk , þá mælti hann við skósvein sinn : " gakk tu , " segir hann , " til úlfs jarls ok drep hann
{'pos': ['AV', 'AP', 'NC', '-', 'AV', 'CC', 'NC', 'VB', '-', 'AV', 'VB', 'PE', 'AP', 'NC', 'DP', '-', '-', 'VB', 'PE', '-', '-', 'VB', 'PD', '-', '-', 'AP', 'NP', 'NC', 'CC', 'VB', 'PE']}
konungr settisk - þá í kyrr - ok sat hann oft á hǫrðalandi ok rogalandi
{'pos': ['NC', 'VB', '-', 'AV', 'AP', 'NC', '-', 'CC', 'VB', 'PE', 'AV', 'AP', 'NP', 'CC', 'NP']}
þat ski

, þá ǽsti hann þann taks til stefnu ok lagaórskurðar
{'pos': ['-', 'AV', 'VB', 'PE', 'DD', 'NC', 'AP', 'NC', 'CC', 'NC']}
en er atla skj var klofinn , þá kastar hann skjald brottnu ok hjó með sverðinu sem ákafast
{'pos': ['CC', 'CS', 'NP', 'NC', 'VB', 'VB', '-', 'AV', 'VB', 'PE', 'NC', 'AV', 'CC', 'VB', 'AP', 'NC', 'CS', 'AJ']}
salt skal hann -
{'pos': ['NC', 'VB', 'PE', '-']}
eigi skyldu þeir ok inir sigrsǽlu vera óþakknǽmir við inn hǽsta sigrvegara
{'pos': ['AV', 'VB', 'DD', 'AV', 'DD', 'AJ', 'VB', 'AJ', 'AP', 'DD', 'AJ', 'NC']}
darius konungr býðr þér til sátta dóttur sína ok þar með í heimanfylgju hennar ǫll lǫnd þau , sem liggja meðal árinnar euphraten ok helles
{'pos': ['UA', 'NC', 'VB', 'PE', 'AP', 'NC', 'NC', 'DP', 'CU', 'AV', 'AP', 'AP', 'NC', 'PE', 'AJ', 'NC', 'DD', '-', 'CU', 'VB', 'AP', 'NC', 'UA', 'CU', 'UA']}
þó gengr sólin um síðir eftir sinni venju
{'pos': ['AV', 'VB', 'NC', 'AP', 'NC', 'AP', 'DP', 'NC']}
skyli konungr láta slíka óhǿfu vinna ? nei , segir hann , aldrigi

en eigi sýnizk mér á þessa leið þat , er þér hafið rǿtt , því at eigi vil ek fyrir lið leggja þat , er ek hefi upp hafit , at auka mitt ríki
{'pos': ['CU', 'AV', 'VB', 'PE', 'AP', 'DD', 'NC', 'DD', '-', 'RP', 'PE', 'VB', 'VB', '-', 'CU', 'AV', 'VB', 'PE', 'AP', 'NC', 'VB', 'DD', '-', 'RP', 'PE', 'VB', 'AV', 'VB', '-', 'CU', 'VB', 'DP', 'NC']}
problème
drápu þeir tolf menn , en rǽntu fé ǫllu , en spilltu því , er þeir máttu eigi með komask
{'pos': ['VB', 'DD', 'NA', 'NC', '-', 'CC', 'VB', 'NC', 'AJ', '-', 'CC', 'VB', 'DD', '-', 'RP', 'DD', 'VB', 'AV', 'VP', 'VB']}
en ef lǫgmanni sýnisk sá réttara hafa er varði , þá liggi jǫrð sem komin er , þar til er lǫgliga verðr undan sótt
{'pos': ['CC', 'CS', 'NC', 'VB', 'DD', 'AJ', 'VB', 'CS', 'VB', '-', 'AV', 'VB', 'NC', 'CS', 'VB', 'VB', '-', 'AV', 'AP', 'CS', 'AV', 'VB', 'AP', 'VB']}
í enni þessa manns var ritat scilicet tetragrammaton
{'pos': ['AP', 'NC', 'DD', 'NC', 'VB', 'VB', 'UA', 'UA']}
allt þat er í kaupangi er gǫrt , þat skal allt at kau

var þat hinn bezti gripr
{'pos': ['VB', 'DD', 'AT', 'AJ', 'NC']}
vér hǫfum hella fyrir hallir ok látum oss þǫrf vinna þat , er náttúran sjǫlf , in fyrsta móðir vár , vill hafa gefit
{'pos': ['PE', 'VB', 'NC', 'AP', 'NC', 'CU', 'VB', 'PE', 'NC', 'VB', 'DD', '-', 'RP', 'NC', 'AJ', '-', 'DD', 'AJ', 'NC', 'DP', '-', 'VB', 'VB', 'VB']}
en þat -
{'pos': ['CC', 'DD', '-']}
en ef maðr tekr fénuð þann í brott , er inn var settr ok vill eigi leysa út at lǫgum , þat -
{'pos': ['CC', 'CS', 'NC', 'VB', 'NC', 'DD', 'AP', 'AV', '-', 'CS', 'AV', 'VB', 'VB', 'CC', 'VB', 'AV', 'VB', 'AV', 'AP', 'NC', '-', 'DD', '-']}
ørtoga tak ok marka tak , er þat -
{'pos': ['NC', 'NC', 'CC', 'NC', 'NC', '-', 'CS', 'DD', '-']}
en er sendimenn koma til óláfs konungs , þá tók hann at búa her sinn ok ǽtlaði þá at herja
{'pos': ['CC', 'CS', 'NC', 'VB', 'AP', 'NP', 'NC', '-', 'AV', 'VB', 'PE', 'IM', 'VB', 'NC', 'DP', 'CC', 'VB', 'AV', 'IM', 'VB']}
bleikt andlit ok hrygglig augu sýna þat , at þú ert miklu mildari óvinr , en

land upp , ok skal einn standa við bryggjusporð , en annarr upp í frá , svá at hverr megi heyra óp annars , ok ǿpa þrysvar ok gangi síðan til skips
{'pos': ['NC', 'AV', '-', 'CC', 'VB', 'AJ', 'VB', 'AP', 'NC', '-', 'CC', 'DQ', 'AV', 'AP', '-', 'AV', 'CS', 'DQ', 'VB', 'VB', 'NC', 'DQ', '-', 'CC', 'VB', 'AV', 'CC', 'VB', 'AV', 'AP', 'NC']}
problème
stýrði ǫðru kveld-ulfr , en ǫðru grímr
{'pos': ['VB', 'NO', 'NP', '-', 'CC', 'NO', 'NP']}
problème
eptir þat batnaði agli
{'pos': ['AP', 'PE', 'VB', 'NP']}
ǽtla ek , at þǽr muni lyktir á falla , at vér munim aldrtila af fá konungi þessum
{'pos': ['VB', 'PE', '-', 'CS', 'DD', 'VB', 'NC', 'VP', 'VB', '-', 'CS', 'PE', 'VB', 'NC', 'AP', 'VB', 'NC', 'DD']}
svá mikil manndómsást byggði brjóst konungs í þann tíma , ef upp tekinn háttr heldisk með honum , at engi ófrǽgð mǽtti saurga hans ina bjǫrtu frǽgð
{'pos': ['AV', 'AJ', 'NC', 'VB', 'NC', 'NC', 'AP', 'DD', 'NC', '-', 'CU', 'AV', 'VB', 'NC', 'VB', 'AP', 'PE', '-', 'CU', 'PI', 'NC', 'VB', 'VB', 'PE'

...

aurum silfrs
{'pos': ['NC', 'NC']}
þegar er indi verða varir við , at alexander konungr nálgazk með her sinn , taka margir þat ráð , því at þeir hafa spurt , at hann hefir ǽ sigr , hvar sem hann snertr við með þat frǿkna lið , er honum fylgir , at þeir fara í mót honum ok friða fyrir sér , gefa borgir sínar ok sjalfa sik í vald hans
{'pos': ['AV', 'RP', 'UA', 'VB', 'AJ', 'AP', '-', 'CU', 'UA', 'NC', 'VB', 'AP', 'NC', 'DP', '-', 'VB', 'AJ', 'DD', 'NC', '-', 'CU', 'DD', 'VB', 'VB', '-', 'CU', 'PE', 'VB', 'AV', 'NC', '-', 'AV', 'CU', 'PE', 'VB', 'AP', 'AP', 'DD', 'AJ', 'NC', '-', 'RP', 'PE', 'VB', '-', 'CU', 'DD', 'VB', 'AP', 'PE', 'CU', 'VB', 'AP', 'PE', '-', 'VB', 'NC', 'DP', 'CU', 'AJ', 'PE', 'AP', 'NC', 'PE']}
problème
hann var auðkenndr af ágǽtligum vápnum þeim , er hann bar
{'pos': ['PE', 'VB', 'VB', 'AP', 'AJ', 'NC', 'DD', '-', 'RP', 'PE', 'VB']}
þeir egill fundu eigi fyrr , er þeir ganga með gǫrðunum , en garðr gekk á aðra hǫnd þeim , ok máttu eigi fram komask
{'pos': ['DD', 'NP',

var þá lið þat allt með agli , er áðr hǫfðu með þeim báðum verit ok ór orrustu hǫfðu komisk
{'pos': ['VB', 'AV', 'NC', 'DD', 'AJ', 'AP', 'NP', '-', 'RP', 'AV', 'VB', 'AP', 'DD', 'PI', 'VB', 'CC', 'AP', 'NC', 'VB', 'VB']}
konungr mǽlti : er sjá hinn mikli maðr skalla-grímr ? hann svaraði ok kvað hann rétt kenna
{'pos': ['NC', 'VB', '-', 'VB', 'DD', 'AT', 'AJ', 'NC', 'NP', '-', 'PE', 'VB', 'CC', 'VB', 'PE', 'AJ', 'VB']}
problème
punda far at hásetu , ok þó at fleiri sé , þá á - -
{'pos': ['NC', 'NC', 'AP', 'NC', '-', 'CC', 'CS', 'AJ', 'VB', '-', 'AV', 'VB', '-', '-']}
problème
en hverr nefndarmaðr er síðar kømr til lǫgþingis nauðsynjalaust en sǿri eru flutt , er sekr mǫrk silfrs
{'pos': ['CC', 'DQ', 'NC', 'CS', 'AV', 'VB', 'AP', 'NC', 'AJ', 'CS', 'NC', 'VB', 'VB', '-', 'VB', 'AJ', 'NC', 'NC']}
alexander konungr lǽtr taka þenna mann ok varðveita til þess , at hann taki slíkt upp , sem hann stundar til , þá er reynt verðr , hvárt hann segir fals eða eigi
{'pos': ['UA', 'NC', 'VB', 'VB', 'D

ok vil ek nú heldr þola dauða en sjá glǿpr fylgi mér til helvítis
{'pos': ['CC', 'VB', 'PE', 'AV', 'AV', 'VB', 'NC', 'CS', 'DD', 'NC', 'VB', 'PE', 'AP', 'NC']}
ok þá verðr mikill ymr í herinum
{'pos': ['CU', 'AV', 'VB', 'AJ', 'NC', 'AP', 'NC']}
kallar síðan hǫfðingja til sín ok skipar til , hvar hverr þeirra skal í fylkingu vera með sínar sveitir þá , er bardaginn tekzk
{'pos': ['VB', 'AV', 'NC', 'AP', 'PE', 'CU', 'VB', 'AP', '-', 'AV', 'PQ', 'DD', 'VB', 'AP', 'NC', 'VB', 'AP', 'DP', 'NC', 'AV', '-', 'RP', 'NC', 'VB']}
ørtogum ok -
{'pos': ['NC', 'CC', '-']}
, ef þess fars missir dǿgri lengr ok verðr farhirðir at því kunnr ok sannr
{'pos': ['-', 'CS', 'DD', 'NC', 'VB', 'NC', 'AV', 'CC', 'VB', 'NC', 'AP', 'DD', 'AJ', 'CC', 'AJ']}
váru þar um hríð
{'pos': ['VB', 'AV', 'AP', 'NC']}
ok þá er alexander spyrr þessi tíðendi , at allr heimr vill til hans lúta , ok sendimenn hǫfðingja bíða hans í babylone , þá skundar hann þangat með her sinn
{'pos': ['CU', 'AV', 'RP', 'UA', 'VB', 'DD', 'NC', '

problème
frá þeim brǿðrum eyvindr ok álfr hétu brǿðr gunnhildar
{'pos': ['AP', 'DD', 'NC', 'NP', 'CC', 'NP', 'VB', 'NC', 'NP']}
ok ef ek hrǽddumk eigi nafn aristotelis meistara míns , þá munda ek freista at líkja nakkvat eftir slíkum stórvirkjum
{'pos': ['CU', 'CU', 'PE', 'VB', 'AV', 'NC', 'UA', 'NC', 'DP', '-', 'AV', 'VB', 'PE', 'VB', 'IM', 'VB', 'PI', 'AP', 'AJ', 'NC']}
varizk þér , at eigi neyti hann af lífstré útan leyfi , svá sem hann neytti af fróðleiks eplum , fyri útan leyfi , at eigi lifi hann eilíf í sínum
{'pos': ['VB', 'PE', '-', 'CS', 'AV', 'VB', 'PE', 'AP', 'NP', 'AP', 'NC', '-', 'AV', 'CS', 'PE', 'VB', 'AP', 'NC', 'NC', '-', 'AP', 'AP', 'NC', '-', 'CS', 'AV', 'VB', 'PE', 'AV', 'AP', 'DP']}
egill fór um nóttina þar til , er skipin váru
{'pos': ['NP', 'VB', 'AP', 'NC', 'AV', 'VP', '-', 'CS', 'NC', 'VB']}
um sýslumannaþing capitulus svá er mǽlt at sýslumenn skulu þing eiga innan þeira - -
{'pos': ['AP', 'NC', 'FW', 'AV', 'VB', 'VB', 'CS', 'NC', 'VB', 'NC', 'VB', 'AP', 'DD',

en þótt manni veiti þungt um stundar sakir , þá kemr þó jafnan logn á bak vindi
{'pos': ['CU', 'CU', 'NC', 'VB', 'AJ', 'AP', 'NC', 'NC', '-', 'AV', 'VB', 'AV', 'AV', 'NC', 'AP', 'NC', 'NC']}
þá kømsk hann eigi til skuldar , nema með váttum , því at engi er skuld lokin , nema með váttum
{'pos': ['AV', 'VB', 'PE', 'AV', 'AP', 'NC', '-', 'CS', 'AP', 'NC', '-', 'CS', 'DQ', 'VB', 'NC', 'VB', '-', 'CS', 'AP', 'NC']}
problème
kveld-ulfr svaraði ok sagði , at konungr hafði til þess orð sent , at ek skylda gerask hans maðr eðr sonr minn annarr hvárr
{'pos': ['NP', 'VB', 'CC', 'VB', '-', 'CS', 'NC', 'VB', 'AP', 'DD', 'NC', 'VB', '-', 'CS', 'PE', 'VB', 'VB', 'PE', 'NC', 'CC', 'NC', 'DP', 'PI']}
problème
þǫkkum þat -
{'pos': ['VB', 'DD', '-']}
þá gekk arinbjǫrn til hans ok spurði , hví hann vǽri svá ókátr , en þótt u hafir misst bróður þíns , þá er þér karlmannligt at bera sik vel , ok skal maðr eftir mann lifa , eðr hvat kveðr þú ? lát mik heyra
{'pos': ['AV', 'VB', 'NP', 'AP', 'PE', 'CC', 'VB', 

In [10]:
!python -m spacy preprocess data-menota-spacy.json

Usage: python -m spacy [OPTIONS] COMMAND [ARGS]...
Try 'python -m spacy --help' for help.

Error: No such command 'preprocess'.


In [11]:
pos_training_data[0]

(['nú',
  'vann',
  'honum',
  'eigi',
  'þǫrf',
  'at',
  'vega',
  'með',
  'tungunni',
  'at',
  'eins'],
 {'pos': ['AV', 'VB', 'PE', 'AV', 'NC', 'IM', 'VB', 'AP', 'NC', 'AP', 'NA']})

don't do it again, it is just to have a first version of config.cfg

In [81]:
# !python -m spacy init fill-config base_config.cfg config.cfg 

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [14]:
nlp = spacy.blank("is")
tagger = nlp.add_pipe("tagger")
tags = []
for item in pos_training_data:
    labels = item[1]["pos"]
    for label in labels:
        tagger.add_label(label)
        tags.append(label)
tags = list(set(tags))
with open ("tag.json", "w", encoding='utf-8') as f:
    print(tags)
    json.dump(tags, f, indent=4)
nlp.to_disk("output/test")

['UA', 'AJ', 'NA', 'EX', 'PE', '-', 'AV', 'RP', 'FW', 'VB', 'DQ', 'FN', 'VP', 'PQ', 'NP', 'CC', 'DD', 'IT', 'NC', 'IM', 'CU', 'DP', 'CS', 'AP', 'AT', 'PD', 'PR', 'PI', 'AQ', 'NO', 'NE']


In [13]:
!python -m spacy train output/test/config.cfg --output ./output --paths.train data/train.spacy --paths.dev data/valid.spacy

[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================


[2021-11-12 23:13:45,461] [INFO] Set up nlp object from config
[2021-11-12 23:13:45,472] [INFO] Pipeline: ['tagger']
[2021-11-12 23:13:45,472] [INFO] Created vocabulary
[2021-11-12 23:13:45,481] [INFO] Finished initializing nlp object
Traceback (most recent call last):
  File "C:\Users\clems\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\clems\AppData\Local\Programs\Python\Python38\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\clems\PycharmProjects\menota-training\venv\lib\site-packages\spacy\__main__.py", line 4, in <module>
    setup_cli()
  File "C:\Users\clems\PycharmProjects\menota-training\venv\lib\site-packages\spacy\cli\_util.py", line 71, in setup_cli
    command(prog_name=COMMAND)
  File "C:\Users\clems\PycharmProjects\menota-training\venv\lib\site-packages\click\core.py", line 1128, in __call__
    return self.main(*args, **kwargs)
  Fil